In [ ]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from importlib import reload
import sys
from scipy.stats import binned_statistic
import warnings
from scipy.optimize import minimize

warnings.filterwarnings("ignore")

In [ ]:
sys.path.insert(1,'../../snowmicropyn')
import snowmicropyn
print(snowmicropyn.__file__)

from snowmicropyn import density_ssa

# Read in mCT data

In [ ]:
def get_mct_frame(site):

    frames = pickle.load(open('../data/microCT/processed_mCT.p', 'rb'))

    mct_df = frames[site]

    mct_df = mct_df.drop_duplicates(subset='height_ave (cm)',keep="first")

    mct_df.set_index('height_ave (cm)', inplace=True)
    
    return(mct_df)

site = '2N13'

mct_df = get_mct_frame(site)

mct_df

In [ ]:
fig,ax = plt.subplots()
ax.plot(mct_df['Equiv. Diam (mm)'],
         mct_df.index)

ax.set_xlabel('Equiv. Diam (mm)')
ax.set_ylabel('Height above snow-soil interface [cm]')

ax.set_title(f'Site {site.upper()}')

plt.show()

# return df to get a look at it

mct_df

# Read SMP data

In [ ]:
p = snowmicropyn.Profile.load(f'../data/SMP/SNEX20_SMP_S19M1172_2N13_20200206.PNT')

p.detect_ground()

# Sensitivity analysis to window size

In [ ]:
C20_ssa_coeffs = [0.57, -18.56, -3.66]

C20_coeffs = {'density': [295.8, 65.1, -43.2, 47.1],
              'ssa':C20_ssa_coeffs,
              'equation':'ssa'}

P15_ssa_coeffs = [0.131, 0.355, 0.0291]

P15_coeffs = {'density':[420.47, 102.47, -121.15, -169.96],
              'ssa':P15_ssa_coeffs,
              'equation':'l_ex'}

C20_window = 1
P15_window = 2.5

In [ ]:
def get_P15_C20(p):

    df_C20 = snowmicropyn.density_ssa.calc(p.samples,
                                            coeff_model=C20_coeffs,
                                            window=C20_window,
                                            overlap=50)

    df_P15 = snowmicropyn.density_ssa.calc(p.samples,
                                            coeff_model=P15_coeffs,
                                            window=P15_window,
                                            overlap=50)

    ###################################################

    df_C20.columns = ['distance', 'Cdensity', 'Cssa']

    df_P15.columns = ['distance', 'Pdensity', 'Pssa']

    ###################################################

    df_combined = pd.merge(left=df_P15, right=df_C20)

    df_combined = df_combined[(df_combined['distance'] < p.ground)]

    df_combined['distance_up'] = (p.ground - df_combined['distance'])/10

    df_combined.set_index('distance_up',inplace=True)

    df_combined.sort_index()

    df_combined.dropna(how='any')
    
    return df_combined

PC_df = get_P15_C20(p)
PC_df

In [ ]:
def resample_PC_to_mCT(mct_df, PC_df):

    for code in ['Cssa', 'Pssa']:

        mct_df[code] = [np.nanmean(PC_df[code][(PC_df.index < u) & (PC_df.index > l)]) 
                          for l, u in zip(mct_df['height_min (cm)'], mct_df['height_max (cm)'])   ]
        
    return mct_df

com_df = resample_PC_to_mCT(mct_df, PC_df)

com_df

In [ ]:
fig,ax = plt.subplots(figsize=(8,5))

ax.plot(com_df['Pssa'], com_df.index,label='P2015')
ax.plot(com_df['Cssa'], com_df.index,label='C2020')

ax.plot(com_df['SSA (m2/kg)'],
         com_df.index,
       label='mCT',
        color='k',
       zorder=5)

ax.set_xlabel('SSA (m2/kg)', fontsize='x-large')
ax.set_ylabel('Height above snow-soil interface [cm]', fontsize='x-large')

ax.set_title(f'Site {site.upper()}', fontsize='x-large')

ax.legend(loc='lower right', fontsize='x-large')

plt.show()

In [ ]:
def get_RMSE(mct_df):

    CRMSE = np.sqrt(np.nanmean(np.square(mct_df['SSA (m2/kg)']-mct_df['Cssa'])))
    PRMSE = np.sqrt(np.nanmean(np.square(mct_df['SSA (m2/kg)']-mct_df['Pssa'])))
    
    return {'CRMSE':CRMSE,
            'PRMSE':PRMSE}

get_RMSE(com_df)

# Now iterate coefficients to find best fit

In [ ]:
def get_ssa(p, coeffs_dict):

    df = snowmicropyn.density_ssa.calc(p.samples,
                                            coeff_model=coeffs_dict,
                                            window=C20_window,
                                            overlap=50)

    df = df[(df['distance'] < p.ground)]

    df['distance_up'] = (p.ground - df['distance'])/10

    df.set_index('distance_up',inplace=True)

    df.sort_index()

    df.dropna(how='any')
    
    return df

def resample_SMP_to_mCT(mct_df, df):
    
    com_df = mct_df.copy()
    
    com_df['ssa'] = [np.nanmean(df['ssa'][(df.index < u) & (df.index > l)]) 
                      for l, u in zip(mct_df['height_min (cm)'], mct_df['height_max (cm)'])   ]
        
    return com_df

In [ ]:
def compare_smp_to_mct(coeffs_list,mct_df,p,eqn='ssa'):
    
    coeffs_dict = {'density': [295.8, 65.1, -43.2, 47.1],
                  'ssa':coeffs_list,
                  'equation':eqn}

    df = get_ssa(p, coeffs_dict)
    
#     print(df)
    
    com_df = resample_SMP_to_mCT(mct_df, df)
    
#     print(com_df)
    
    return(com_df)

def calc_RMSE(coeffs_list,mct_df,p,eqn='ssa'):
    
    com_df = compare_smp_to_mct(coeffs_list,mct_df,p,eqn)
    
    RMSE = np.sqrt(np.nanmean(np.square(com_df['SSA (m2/kg)']-com_df['ssa'])))
    
    print(RMSE)
    
    return RMSE

In [ ]:
site = '2N13'

mct_df = get_mct_frame(site)

RMSE = calc_RMSE(C20_ssa_coeffs,mct_df,p)

In [ ]:
initial_guess = C20_ssa_coeffs

min_output = minimize(fun=calc_RMSE,
                     x0=initial_guess,
                     args=(mct_df, p))

# Plot the results of the minimization exercise

In [ ]:
ssa_coeffs_min = list(min_output.x)

min_df = compare_smp_to_mct(ssa_coeffs_min,mct_df,p)

#############

C20_coeffs = [0.57, -18.56, -3.66]

C20_df = compare_smp_to_mct(C20_coeffs,mct_df,p)

# ################

P15_coeffs = [0.131, 0.355, 0.0291]

P15_df = compare_smp_to_mct(P15_coeffs,mct_df,p,eqn='l_ex')

In [ ]:
plt.plot(min_df['ssa'],min_df.index,label='minimized coeffs')

plt.plot(P15_df['ssa'],min_df.index,label='P2015')

plt.plot(C20_df['ssa'],min_df.index,label='C2020')

plt.plot(min_df['SSA (m2/kg)'],min_df.index,label='micro-CT')

plt.xlabel('SSA (m2/kg)', fontsize='x-large')
plt.ylabel('Height above snow-\nsoil interface [cm]', fontsize='x-large')

plt.legend()